In [2]:
import pandas as pd 
import keras 
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.optimizers
from sklearn.model_selection import GridSearchCV

In [3]:
previsores = pd.read_csv('entradas-breast.csv')
classe = pd.read_csv('saidas-breast.csv')

# Trocar parametros fixos pelos parametros que devem ser informado na entrada na função, na tentantiva de fazer uma grade de pesquisa de todas as possibilidade dos parâmetros para encontrar a combinação que retorne o melhor resultado.

## Observe que a camada de saída nao muda, pois ele seguirá sendo um retorno binario, e uma unica resposta, como se trata de uma classificação a função de ativação segue sendo uma sigmoide.

In [3]:
def criarRede(optimizer, loos, kernel_initializer, activation, neurons):
    classificador = Sequential()
    classificador.add(Dense(units = neurons, activation = activation,
                           kernel_initializer = kernel_initializer, input_dim = 30))
    classificador.add(Dropout(0.2))
    classificador.add(Dense(units = neurons, activation = activation,
                           kernel_initializer = kernel_initializer))
    classificador.add(Dropout(0.2))
    classificador.add(Dense(units = 1, activation = 'sigmoid'))
    classificador.compile(optimizer = optimizer, loss = loos,
                         metrics = ['binary_accuracy'])
    return classificador 

# Aqui colocamos varias possibilidade de parâmetros para que a pesquisa seja feita, não somente os parametros da função 'criarRede' mas também as referemtes 'batch_size' e 'epochs' referente ao calculo dos pesos da rede neural.

# Observe que um bom ponto médio para começarmos a testar o numero de neuronios é aquela formula falada anteriormente, onde soma-se as features mais a saida, no caso eram 30 features(camada de entrada) + 1 saida(resposta binaria, cancer é beningno ou maligno), dando 15.5 mas arredondamos para 16, logo podemos especular valores de neuronios da camada oculta em torno desse valor.

In [4]:
classificador = KerasClassifier(build_fn = criarRede)

parametros = {'batch_size': [10,30],
              'epochs': [50,100],
              'optimizer': ['adam', 'sgd'],
              'loos': ['binary_crossentropy', 'hinge'],
              'kernel_initializer': ['random_uniform', 'normal'],
              'activation': ['relu', 'tanh'],
              'neurons': [16,8]}

# Aqui colocamos tudo no Grid Search CV

In [ ]:
grid_search = GridSearchCV(estimator = classificador,
                          param_grid = parametros,
                          scoring = 'accuracy',
                          cv = 5)

grid_search = grid_search.fit(previsores, classe)

# Observamos os melhores parametros, e se um parâmetro númerico atingiu o maior valor dos que foram colocados para serem testados, vale a pena refazer o teste com valores maiores, pois pode estar sendo indicado que irá achar o melhor resultado em valores maiores. Mesma lógica se for o menor valor. Entaõ deve-se observar os melhores parametros para readequar, se necessario, os valores a serem testados e refazer o teste.

In [ ]:
melhores_parametros = grid_search.best_params_

In [ ]:
melhor_precisao = grid_search.best_score_